In [1]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="NW0OR7CAuzrGbEzhXpzF")
project = rf.workspace("project-one-7x932").project("pressed_key_check")
version = project.version(6)
dataset = version.download("coco")
                

loading Roboflow workspace...
loading Roboflow project...
Exporting format coco in progress : 85.0%
Version export complete for coco format



Extracting Dataset Version Zip to pressed_key_check-6 in coco:: 100%|███████████████| 732/732 [00:01<00:00, 408.98it/s]


In [2]:
!pip install supervision opencv-python-headless scikit-learn matplotlib

In [5]:
import json
from collections import defaultdict

with open("pressed_key_check-6/valid/_annotations.coco.json") as f:
    data = json.load(f)

# Create a mapping from class id to name
categories = {c['id']: c['name'] for c in data['categories']}

# Ground truth dictionary: {image_id: [label1, label2, ...]}
ground_truth = defaultdict(list)

for ann in data['annotations']:
    image_id = ann['image_id']
    category_id = ann['category_id']
    ground_truth[image_id].append(categories[category_id])


In [7]:
import supervision as sv
from roboflow import Roboflow
import os
import cv2

# Load images
valid_images_path = "pressed_key_check-6/valid"
images = [f for f in os.listdir(valid_images_path) if f.endswith(".jpg") or f.endswith(".png")]

# Load your model
rf = Roboflow(api_key="NW0OR7CAuzrGbEzhXpzF")
project = rf.workspace("project-one-7x932").project("pressed_key_check")
model = project.version(4).model

predictions = {}

for image_file in images:
    result = model.predict(os.path.join(valid_images_path, image_file), confidence=40, overlap=30).json()
    labels = [prediction["class"] for prediction in result["predictions"]]
    predictions[image_file] = labels


loading Roboflow workspace...
loading Roboflow project...


In [9]:
y_true = []
y_pred = []

# Map Roboflow image names to COCO IDs
image_id_lookup = {img["file_name"]: img["id"] for img in data["images"]}

for img_name in predictions:
    if img_name not in image_id_lookup:
        continue
    img_id = image_id_lookup[img_name]
    gt_labels = ground_truth[img_id]
    pred_labels = predictions[img_name]
    
    # If multiple labels, match pairwise for confusion matrix (or adapt logic)
    for gt, pred in zip(gt_labels, pred_labels):
        y_true.append(gt)
        y_pred.append(pred)




In [11]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

labels = sorted(list(set(y_true + y_pred)))

cm = confusion_matrix(y_true, y_pred, labels=labels)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=labels)

plt.figure(figsize=(10, 8))
disp.plot(cmap='Blues', xticks_rotation=45)
plt.title("Confusion Matrix - Pressed Key Detection")
plt.show()


ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject